In [ ]:
from langchain_google_vertexai import VertexAI
llm = VertexAI(model_name="gemini-1.5-pro")
print(llm.invoke("Which question can you answer?"))

In [ ]:
task = "Write a poem about Google CLoud and LangCHain"
for chunk in llm.stream(task):
    print(chunk)

In [ ]:
from langchain_google_vertexai import HarmBlockThreshold, HarmCategory

for chunk in llm.stream(
    "Write a poem about Sango Ota and Nigeria",
    temperature=0.9,
    max_output_tokens=100,
    stop=["."],
    safety_settings={
        HarmCategory.HARM_CATEGORY_HATE_SPEECH
        : HarmBlockThreshold.BLOCK_LOW_AND_ABOVE
    }
):
    print(chunk)

### Templating

In [ ]:
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
prompt_template = PromptTemplate.from_template(
    """Extract {entities} entities from the item description: \n{description}\n.
    Answer with a valid json as an output."""
)
chain = prompt_template | llm | JsonOutputParser()

In [ ]:
description = """It costs #=900 pounds with 16GB of RAM. Buy Pixel 8 and get up to £355 back with eligible phone trade-in. \n
Offer starts on 05/02/2025 at 00:00 GMT and ends on 12/03/2025 at 23:59 GMT.  \n
Typical representative amount: £355 for """
result = chain.invoke(
    {"entities": "price, RAM", "description": description}
)
print(result)

### Chat models

In [ ]:
from langchain_core.messages import BaseMessage, HumanMessage
message = BaseMessage(
    content="Hi, how are you?",
    type="human",
    additional_kwargs={"chapter": 2}
)

In [ ]:
from langchain_google_vertexai import ChatVertexAI
chat_model = ChatVertexAI(model_name="gemini-1.5-pro")
message = HumanMessage(content="What is new today?")
answer = chat_model.invoke([message])
print(answer.content)

In [ ]:
message2 = HumanMessage(content="What is the cost of two pineapples from ASDA?")
answer2 = chat_model.invoke(
    [message, answer, message2], temperature=0.9
)
print(answer2.content)

In [ ]:
print(answer.response_metadata["usage_metadata"])

In [ ]:
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate
)
from langchain_core.messages import SystemMessage
chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=(
                "You are a helpful assistant that helps extract entities from product descriptions."
                "You always respond in a json format."
            )
        ),
        HumanMessagePromptTemplate.from_template(
            "Extract the following entities: \n {entities} \n from the item's description: \n {description}"
        ),
    ]
)

In [ ]:
chain = chat_template | chat_model | JsonOutputParser()
result = chain.invoke({"entities": "price, RAM",
                       "description": description})

In [ ]:
result

In [ ]:
from langchain_google_vertexai.callbacks import VertexAICallbackHandler
handler = VertexAICallbackHandler()
config = {
    "callbacks": [handler]
}
result = chain.invoke(
    {"entities": "price, RAM",
     "description": description}, config=config
)
print(handler.prompt_tokens)

In [ ]:
fast_llm = VertexAI(
    model_name="gemini-1.5-flash", max_output_tokens=2048)
print(fast_llm.invoke("Generate a python script to sort a list of integer numbers. FAST!"))

In [ ]:
import time

def merge_sort(arr):
  """
  Implements the merge sort algorithm for efficient sorting.

  Args:
    arr: The list of integers to sort.

  Returns:
    The sorted list.
  """
  if len(arr) <= 1:
    return arr
  mid = len(arr) // 2
  left_half = merge_sort(arr[:mid])
  right_half = merge_sort(arr[mid:])
  return merge(left_half, right_half)

def merge(left, right):
  """
  Merges two sorted lists into a single sorted list.

  Args:
    left: The first sorted list.
    right: The second sorted list.

  Returns:
    The merged sorted list.
  """
  result = []
  i = j = 0
  while i < len(left) and j < len(right):
    if left[i] <= right[j]:
      result.append(left[i])
      i += 1
    else:
      result.append(right[j])
      j += 1
  result += left[i:]
  result += right[j:]
  return result

# Generate a large list of random integers
import random
numbers = [random.randint(0, 100000) for _ in range(100000)]

# Measure sorting time using merge sort
start_time = time.time()
sorted_numbers = merge_sort(numbers.copy())
end_time = time.time()
print("Merge Sort time:", end_time - start_time)

# Measure sorting time using Python's built-in sort
start_time = time.time()
numbers.sort()  # Python's built-in Timsort
end_time = time.time()
print("Python's built-in sort time:", end_time - start_time)

In [ ]:
# from langchain_google_vertexai import VertexAIModelGarden
# llama_endpoint_id = "meta/llama-3.1-8b-instruct-maas"
# project = "gcp-langchain-450916"
# location = "us-central1"
# llama_model = VertexAIModelGarden(
#     endpoint_id=llama_endpoint_id,
#     project=project,
#     location=location,
# )
# output = llama_model.invoke(["How much is 2+2"])

### Prompt Engineering

In [ ]:
instruction = (
    """---INSTRUCTION--- \nYou are an intelligent assistant that helps marketers 
    write great copy for campaigns on our website,     which sells premium ceiling
      fans to design-conscious customers. Please create campaign copy 
      (a slogan, a tagline, a short description, and three calls-to-action) based on keywords. 
      Use the information from your context to choose the right products to advertise.
        Follow the examples below to ensure that you follow company branding standards.\n"""
)

examples = [
    {
        "keywords": """best fan for hot summer days, powerful, cozy, wood tone, enjoy cold drink""",
        "response": (
            "Slogan:  Breeze 4000: Feel the Difference.\n"
            "Tagline: Design, Comfort, Performance – The Ultimate Summer Upgrade.\n"
            "Short Description:  Beat the heat in style with the Breeze 4000. Its sleek wood-tone design and "
            "whisper-quiet operation create the perfect oasis for enjoying a cool drink on those hot summer days.\n"
            "Call to action: 1/ Experience the Breeze 4000 difference today.  (Emphasizes the unique qualities)\n"
            "2/ Upgrade your summer. Shop the Breeze 4000 now. (Creates a sense of urgency)\n"
            "3/ Find your perfect Breeze 4000 style. (Focus on design and personalization)"
        )
    },
]

prompt_template = "---CONTEXT---\n{context}\n------KEYWORDS FOR CREATING COPY---\n{keywords}\n---EXAMPLES---\n{examples}"
context = [
    {
        "name": "Whirlwind BreezeMaster 3000",
        "performanceRating": "high",
        "outdoor": True,
        "powerSource": "electric",
        "price": 249.99
    }
]
keywords = "best fan for dry heat, powerful, outdoor, porch, affordable"

In [ ]:
prompt_template

In [ ]:
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate
example_prompt = PromptTemplate(
    input_variables=["keywords", "response"],
    template="Example keywords: \n{keywords}\nExample response:\n{response}"
)
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=instruction,
    suffix="---CONTEXT---\n{context}\n---KEYWORDS FOR CREATING COPY---\n{keywords}\n",
    input_variables=["context", "keywords"],
)

In [ ]:
llm = VertexAI(model_name="gemini-pro")
response = (prompt | llm).invoke({"context": context, "keywords": keywords})
print(response)